# quantification subclasses

> Extends the classes defined in core for the quantification-specific requirements (findmycells.quantification.specs)

- order: 11

In [ ]:
#| default_exp quantification/specs

In [ ]:
#| export

from typing import Dict, List
from skimage import io

from findmycells.core import ProcessingObject, ProcessingStrategy
from findmycells.configs import DefaultConfigs
from findmycells import utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class QuantificationStrategy(ProcessingStrategy):
    
    """
    Note for developers: When implementing a new quantification strategy, remember to add
    the following line at the end of the ".run()" method, to ensure that the quantification
    results are added to the database: 
    
    quantification_object = self._add_quantification_results_to_database(quantification_object = quantification_object, 
    results = quantification_results)
    """
    
    @property
    def processing_type(self):
        return 'quantification'


    def _add_quantification_results_to_database(self, quantification_object: ProcessingObject, results: Dict) -> ProcessingObject:
        if hasattr(quantification_object.database, 'quantification_results') == False:
            setattr(quantification_object.database, 'quantification_results', {})
        if self.__class__.__name__ not in quantification_object.database.quantification_results.keys():
            quantification_object.database.quantification_results[self.__class__.__name__] = {}
        quantification_object.database.quantification_results[self.__class__.__name__][quantification_object.file_id] = results
        return quantification_object

In [ ]:
#| export

class QuantificationObject(ProcessingObject):
    
    """
    Extending the `ProcessingObject` base class for quantification as processing subtype.
    """
    
    @property
    def processing_type(self) -> str:
        return 'quantification'
    
    @property
    def widget_names(self):
        widget_names = {'overwrite': 'Checkbox',
                        'autosave': 'Checkbox',
                        'show_progress': 'Checkbox'}
        return widget_names

    @property
    def descriptions(self):
        descriptions = {'overwrite': 'overwrite previously processed files',
                        'autosave': 'autosave progress after each file',
                        'show_progress': 'show progress bar and estimated computation time'}
        return descriptions
    
    @property
    def tooltips(self):
        return {} 
    
    @property
    def default_configs(self) -> DefaultConfigs:
        default_values = {'overwrite': False,
                          'autosave': True,
                          'show_progress': True}
        valid_types = {'overwrite': [bool],
                       'autosave': [bool],
                       'show_progress': [bool]}
        default_configs = DefaultConfigs(default_values = default_values,
                                         valid_types = valid_types)
        return default_configs
    
    
    def _processing_specific_preparations(self) -> None:
        self.file_id = self.file_ids[0]
        self.segmentations_per_area_roi_id = self._load_postprocessed_segmentations()


    def _load_postprocessed_segmentations(self) -> Dict:
        segmentations_per_area_roi_id = {}
        quantified_segmentations_dir_path = self.database.project_configs.root_dir.joinpath(self.database.quantified_segmentations_dir)
        for elem in quantified_segmentations_dir_path.iterdir():
            if elem.is_dir():
                matching_filepaths = [filepath for filepath in utils.list_dir_no_hidden(elem) if filepath.name.startswith(self.file_id)]
                if len(matching_filepaths) > 0:
                    area_roi_id = elem.name
                    segmentations_per_area_roi_id[area_roi_id] = utils.load_zstack_as_array_from_single_planes(path = elem, file_id = self.file_id)
        return segmentations_per_area_roi_id


    def _add_processing_specific_infos_to_updates(self, updates: Dict) -> Dict:
        return updates

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()